In [1]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from keras.callbacks import ReduceLROnPlateau
from skimage import exposure
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate
from keras.models import Model

In [2]:
# Function to get training data
def get_training_data(data_dir):
    data = []
    labels = ['PNEUMONIA', 'NORMAL']
    img_size = 150
    for label in labels:
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                resized_arr = cv2.resize(img_arr, (img_size, img_size))  # Reshaping images to preferred size
                data.append([resized_arr, class_num])
            except Exception as e:
                print(e)
    return np.array(data)

In [3]:
# Load training, testing, and validation data
train = get_training_data('/Users/deva/Desktop/DEVA/CSUN/Sem_3/Research/chest_xray/train')
test = get_training_data('/Users/deva/Desktop/DEVA/CSUN/Sem_3/Research/chest_xray/test')
val = get_training_data('/Users/deva/Desktop/DEVA/CSUN/Sem_3/Research/chest_xray/val')


/var/folders/0m/h9z_23v10pj2hclcsszt6v2c0000gn/T/ipykernel_79119/2024085280.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(data)


In [4]:
# Separate features and labels
x_train, y_train = zip(*train)
x_test, y_test = zip(*test)
x_val, y_val = zip(*val)

In [5]:
# Convert to numpy arrays
x_train = np.array(x_train)
x_test = np.array(x_test)
x_val = np.array(x_val)
y_train = np.array(y_train)
y_test = np.array(y_test)
y_val = np.array(y_val)

In [6]:
# Normalize the data
x_train = x_train / 255.0
x_test = x_test / 255.0
x_val = x_val / 255.0

In [7]:
# Define the autoencoder model for denoising and enhancing image quality

def autoencoder_model(input_shape=(150, 150, 1)):
    inputs = Input(shape=input_shape)

  # Encoder 
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    pool1 = MaxPooling2D((2, 2))(conv1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)
    pool2 = MaxPooling2D((2, 2))(conv2)
    

  # Decoder
    conv3 = Conv2D(64, (3, 3), activation='relu', padding='same')(pool2)
    up1 = UpSampling2D((2, 2))(conv3)
    conv4 = Conv2D(32, (3, 3), activation='relu', padding='same')(up1)
    up2 = UpSampling2D((2, 2))(conv4)  # Upsample by a factor of 2
    decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(up2)

    autoencoder = Model(inputs, decoded)
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
    return autoencoder

# Create and train the autoencoder model
autoencoder = autoencoder_model()
autoencoder.fit(x_train, x_train, epochs=10, batch_size=32, validation_data=(x_val, x_val))



Epoch 1/10


ValueError: Arguments `target` and `output` must have the same shape. Received: target.shape=(32, 150, 150), output.shape=(32, 300, 300)

In [ ]:
# Preprocess images using the trained autoencoder
x_train_preprocessed = autoencoder_model.predict(x_train)
x_test_preprocessed = autoencoder_model.predict(x_test)
x_val_preprocessed = autoencoder_model.predict(x_val)

In [ ]:
# Define the U-Net model for segmentation and classification
def unet_model(input_shape=(150, 150, 1)):
    inputs = Input(shape=input_shape)
    
    # Encoder
    conv1 = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    pool1 = MaxPooling2D((2, 2))(conv1)
    conv2 = Conv2D(64, (3, 3), activation='relu', padding='same')(conv1)  # Set padding to 'same'
    pool2 = MaxPooling2D((2, 2))(conv2)
    
    # Bridge
    conv3 = Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    
    # Decoder
    up1 = UpSampling2D((2, 2))(conv3)
    up1 = concatenate([conv2, up1], axis=-1)
    conv4 = Conv2D(64, (3, 3), activation='relu', padding='same')(up1)
    up2 = UpSampling2D((2, 2))(conv4)
    up2 = concatenate([conv1, up2], axis=-1)
    conv5 = Conv2D(32, (3, 3), activation='relu', padding='same')(up2)
    
    # Output
    outputs = Conv2D(1, (1, 1), activation='sigmoid')(conv5)
    
    model = Model(inputs, outputs)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [ ]:
# Create and train the U-Net model
unet = unet_model()
history = unet.fit(x_train_preprocessed, y_train, epochs=12, batch_size=64, validation_data=(x_val_preprocessed, y_val))

# Evaluate the U-Net model
test_loss, test_acc = unet.evaluate(x_test_preprocessed, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)


In [ ]:
# Visualize training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
# Make predictions using the U-Net model
predictions = unet.predict(x_test_preprocessed)
predictions = (predictions > 0.5).astype(int)

In [ ]:
# Generate classification report and confusion matrix
print(classification_report(y_test, predictions, target_names=['PNEUMONIA', 'NORMAL']))
cm = confusion_matrix(y_test, predictions)
print("Confusion Matrix:")
print(cm)